In [5]:
# from google.colab import drive
# drive.mount("/content/gdrive")

## Load the dataset
Note: we will need to turn the label values into -1 and 1.

In [6]:
import pandas as pd
train = pd.read_csv("/Users/divytripathy/PycharmProjects/Machine Learning/Perceptron/bank-note/train.csv", names=['variance', 'skewness', 'curtosis', 'entropy', 'label'])
print(train.head())

   variance  skewness  curtosis  entropy  label
0  3.848100  10.15390  -3.85610 -4.22280      0
1  4.004700   0.45937   1.36210  1.61810      0
2 -0.048008  -1.60370   8.47560  0.75558      0
3 -1.266700   2.81830  -2.42600 -1.88620      1
4  2.203400   5.99470   0.53009  0.84998      0


In [7]:
train['label'] = train['label'].apply(lambda x: 1 if x == 1 else -1)
print(train.head())

   variance  skewness  curtosis  entropy  label
0  3.848100  10.15390  -3.85610 -4.22280     -1
1  4.004700   0.45937   1.36210  1.61810     -1
2 -0.048008  -1.60370   8.47560  0.75558     -1
3 -1.266700   2.81830  -2.42600 -1.88620      1
4  2.203400   5.99470   0.53009  0.84998     -1


In [8]:
test = pd.read_csv("/Users/divytripathy/PycharmProjects/Machine Learning/Perceptron/bank-note/test.csv", names=['variance', 'skewness', 'curtosis', 'entropy', 'label'])
print(test.head())

   variance  skewness  curtosis   entropy  label
0   3.83840    6.1851  -2.04390 -0.033204      0
1   2.85210    9.1710  -3.64610 -1.204700      0
2   5.24180   10.5388  -4.11740 -4.279700      0
3  -2.26230   12.1177   0.28846 -7.758100      0
4   0.55298   -3.4619   1.70480  1.100800      1


In [9]:
test['label'] = test['label'].apply(lambda x: 1 if x == 1 else -1)
print(test.head())

   variance  skewness  curtosis   entropy  label
0   3.83840    6.1851  -2.04390 -0.033204     -1
1   2.85210    9.1710  -3.64610 -1.204700     -1
2   5.24180   10.5388  -4.11740 -4.279700     -1
3  -2.26230   12.1177   0.28846 -7.758100     -1
4   0.55298   -3.4619   1.70480  1.100800      1


Our Class:

In [10]:
import numpy as np
from sklearn.utils import shuffle

class Perceptron:
  def standard_perceptron(self, X: np.ndarray, Y: np.ndarray, rate: int=1, epoch: int=1):
    """
    This only works when the output is binary in the form of -1 or 1.
    """
    weights: np.ndarray = np.zeros(len(X[0]))
    copy_X = X[:]
    copy_Y = Y[:]
    for _ in range(epoch):
      copy_X, copy_Y = shuffle(copy_X, copy_Y)
      for row, y in zip(copy_X, copy_Y):
        y_pred = np.sign(np.dot(weights, row)).astype(np.int64)
        if y != y_pred:
          weights = weights + rate * (y * row)
    return weights

  def voted_perceptron(self, X: np.ndarray, Y: np.ndarray, rate: int=1, epoch: int=1):
    """
    This only works when the output is binary in the form of -1 or 1.
    returns weights in the form of a list of tuples (weights_i, c_i)
    """
    weights_array: list[tuple] = []
    weights: np.ndarray = np.zeros(len(X[0]))
    c: int = 0

    copy_X = X[:]
    copy_Y = Y[:]

    for _ in range(epoch):
      copy_X, copy_Y = shuffle(copy_X, copy_Y)
      for row, y in zip(copy_X, copy_Y):
        y_pred = np.sign(np.dot(weights, row)).astype(np.int64)
        if y != y_pred:
          weights_array.append((weights, c))
          weights = weights + rate * (y * row)
          c = 1
        else:
          c += 1
    return weights_array

  def average_perceptron(self, X: np.ndarray, Y: np.ndarray, rate: int=1, epoch: int=1):
    """
    This only works when the output is binary in the form of -1 or 1.
    """
    weights: np.ndarray = np.zeros(len(X[0]))
    a: np.ndarray = np.zeros(len(X[0]))
    copy_X = X[:]
    copy_Y = Y[:]
    for _ in range(epoch):
      copy_X, copy_Y = shuffle(copy_X, copy_Y)
      for row, y in zip(copy_X, copy_Y):
        y_pred = np.sign(np.dot(weights, row)).astype(np.int64)
        if y != y_pred:
          weights = weights + rate * (y * row)
        a += weights
    return a

  @staticmethod
  def voted_perceptron_predict(datapoint: np.ndarray, weights: list):
    predictions: float = 0
    for i in range(len(weights)):
      predictions += np.dot(weights[i][0], datapoint) * weights[i][1]
    return np.sign(predictions).astype(np.int32)

  @staticmethod
  def bias_trick(X: np.ndarray):
    """
    Used to add 1 to the input array
    """
    output: list[np.ndarray] = []
    for i in range(len(X)):
      output.append(np.append(X[i], 1))
    return np.array(output)

Quick way to check for if bias_trick works.

In [11]:
array_ex: np.ndarray = np.array([np.array([1, 3, 3]), np.array([2, 3, 3]), np.array([3, 3, 4]), np.array([4, 3, 1])])
print(Perceptron.bias_trick(array_ex))

[[1 3 3 1]
 [2 3 3 1]
 [3 3 4 1]
 [4 3 1 1]]


We will look at the accuracy of the standard perceptron.

In [12]:
print(Perceptron().standard_perceptron(X = Perceptron.bias_trick(train.drop('label', axis=1).values), Y = train['label'].values, epoch=10))

[-63.17745  -40.17069  -48.991205  -7.601841  52.      ]


Lets make test predictions.

In [13]:
def accuracy_standard_perceptron(y_test: np.ndarray, x_test: np.ndarray, weights: np.ndarray):
  num_observations: int = len(y_test)
  correct: int = 0
  for i in range(num_observations):
    prediction = np.sign(np.dot(weights, x_test[i]))
    if prediction == y_test[i]:
      correct += 1
  return correct / num_observations

In [14]:
weights = Perceptron().standard_perceptron(X = Perceptron.bias_trick(train.drop('label', axis=1).values), Y = train['label'].values, epoch=10)
print("standard weights: ", weights)
print("train accuracy: ", accuracy_standard_perceptron(train['label'].values, Perceptron.bias_trick(train.drop('label', axis=1).values), weights))
print("test accuracy: ", accuracy_standard_perceptron(test['label'].values, Perceptron.bias_trick(test.drop('label', axis=1).values), weights))

standard weights:  [-62.460361 -43.256255 -34.567106 -13.640989  55.      ]
train accuracy:  0.9724770642201835
test accuracy:  0.97


Quick check of if everything is as expected.

In [15]:
weights = Perceptron().voted_perceptron(X = Perceptron.bias_trick(train.drop('label', axis=1).values), Y = train['label'].values, epoch=10)
print(len(weights))

258


Lets test predictions.

In [16]:
prediction = Perceptron.voted_perceptron_predict(Perceptron.bias_trick(train.drop('label', axis=1).values)[0], weights)
print(prediction, train['label'].values[0])

-1 -1


Lets test accuracy.

In [17]:
def accuracy_voted_perceptron(y_test: np.ndarray, x_test: np.ndarray, weights: list[tuple]):
  num_observations: int = len(y_test)
  correct: int = 0
  for i in range(num_observations):
    prediction = Perceptron.voted_perceptron_predict(x_test[i], weights)
    if prediction == y_test[i]:
      correct += 1
  return correct / num_observations

We need to print the count of the weights along with the weights for the voted perceptron.

In [18]:
weights = Perceptron().voted_perceptron(X = Perceptron.bias_trick(train.drop('label', axis=1).values), Y = train['label'].values, epoch=10)
print("train accuracy: ", accuracy_voted_perceptron(train['label'].values, Perceptron.bias_trick(train.drop('label', axis=1).values), weights))
print("test accuracy: ", accuracy_voted_perceptron(test['label'].values, Perceptron.bias_trick(test.drop('label', axis=1).values), weights))

train accuracy:  0.9873853211009175
test accuracy:  0.986


In [19]:
for weight, count in weights:
  print(count, weight)

0 [0. 0. 0. 0. 0.]
1 [-1.3995  -1.9162   2.5154   0.59912  1.     ]
5 [-3.3335     -1.91619072 -2.3006      0.93879     0.        ]
3 [-5.3401     -8.63519072  6.7156      1.038775    1.        ]
17 [-7.5017     -1.75479072 -1.4361      1.119823    0.        ]
6 [-7.78866     1.42360928 -5.0128     -2.069777    1.        ]
13 [-9.20606    -0.82989072 -3.4948     -1.449967    2.        ]
1 [ -6.75876    -13.45459072  -4.23053      6.211233     1.        ]
2 [ -7.53871    -10.22239072  -7.51253      3.110833     2.        ]
15 [-11.16141     -6.22659072  -7.87098     -0.793867     3.        ]
2 [-14.43061    -18.96719072   7.68632     -0.935687     4.        ]
9 [-16.87921    -12.64969072  -0.27688     -1.141707     3.        ]
3 [-18.32711     -7.77029072  -8.61968      0.966893     2.        ]
5 [-17.42304     -4.39949072 -13.11838     -2.729607     3.        ]
3 [-14.77514    -14.53689072 -11.78738      2.741093     2.        ]
2 [-14.709011   -12.04549072 -14.72748      2.119533     

Lets check the average perceptron accuracy.

In [20]:
def accuracy_average_perceptron(y_test: np.ndarray, x_test: np.ndarray, weights: np.ndarray):
  num_observations: int = len(y_test)
  correct: int = 0
  for i in range(num_observations):
    prediction = np.sign(np.dot(weights, x_test[i]))
    if prediction == y_test[i]:
      correct += 1
  return correct / num_observations

In [21]:
weights = Perceptron().average_perceptron(X = Perceptron.bias_trick(train.drop('label', axis=1).values), Y = train['label'].values, epoch=10)
print("average perceptron weights: ", weights)
print("train accuracy: ", accuracy_average_perceptron(train['label'].values, Perceptron.bias_trick(train.drop('label', axis=1).values), weights))
print("test accuracy: ", accuracy_average_perceptron(test['label'].values, Perceptron.bias_trick(test.drop('label', axis=1).values), weights))

average perceptron weights:  [-385681.46765    -253228.58670999 -252842.27964101  -77965.229805
  349363.        ]
train accuracy:  0.9850917431192661
test accuracy:  0.986
